# Projet : Arbres de Decision, Extensions et Applications

**Annee universitaire 2024-2025**

---

## Table des matieres
1. Partie 1 - Revision des concepts et experiences numeriques
2. Partie 2 - Implementation d'un mini-arbre de decision "from scratch"
3. Partie 3 - Extensions : sur-apprentissage, elagage et forets aleatoires
4. Partie 4 - Application metier et interpretation


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')


---
# Partie 1 - Revision des concepts et experiences numeriques

## 1.1 Rappel theorique

### Classification supervisee
La classification supervisee est une technique d'apprentissage automatique ou:
- On dispose d'un ensemble d'exemples (x, y) avec x les attributs et y la classe
- Le but est d'apprendre une fonction f telle que f(x) predit y
- L'apprentissage est "supervise" car on connait les vraies classes

### Arbre de decision
Un arbre de decision est une structure hierarchique composee de:
- **Noeuds internes**: contiennent un test sur un attribut (ex: age > 30?)
- **Branches**: representent les resultats du test
- **Feuilles**: contiennent la prediction de classe

### Mesures d'impurete
L'impurete mesure l'heterogeneite des classes dans un noeud. Soit p la proportion de la classe positive:

1. **Indice de Gini**: $Gini(p) = 2p(1-p)$ ou $Gini = 1 - \sum p_i^2$
2. **Entropie**: $H(p) = -p\log_2(p) - (1-p)\log_2(1-p)$
3. **Erreur de classification**: $E(p) = 1 - \max(p, 1-p)$


## 1.3 Implementation des fonctions d'impurete

In [ ]:
def gini(counts):
    """Calcule l'indice de Gini pour une distribution de classes."""
    counts = np.array(counts)
    total = np.sum(counts)
    if total == 0:
        return 0
    proportions = counts / total
    return 1 - np.sum(proportions ** 2)

def entropy(counts):
    """Calcule l'entropie pour une distribution de classes."""
    counts = np.array(counts)
    total = np.sum(counts)
    if total == 0:
        return 0
    proportions = counts / total
    proportions = proportions[proportions > 0]
    return -np.sum(proportions * np.log2(proportions))

def classification_error(counts):
    """Calcule l'erreur de classification pour une distribution de classes."""
    counts = np.array(counts)
    total = np.sum(counts)
    if total == 0:
        return 0
    proportions = counts / total
    return 1 - np.max(proportions)


## 1.2 Exemples numeriques d'impurete

In [ ]:
# Definition des cas de test
test_cases = [
    ("Cas equilibre (10/10)", [10, 10]),
    ("Cas tres pur (18/2)", [18, 2]),
    ("Cas 9/1", [9, 1]),
    ("Cas 5/5", [5, 5]),
    ("Cas 1/9", [1, 9]),
]

print(f"{'Cas':<25} {'Gini':>10} {'Entropie':>10} {'Err. Class':>12}")
print("-" * 60)

results = []
for name, counts in test_cases:
    g = gini(counts)
    e = entropy(counts)
    c = classification_error(counts)
    results.append((name, counts, g, e, c))
    print(f"{name:<25} {g:>10.4f} {e:>10.4f} {c:>12.4f}")


### Visualisation des fonctions d'impurete

In [ ]:
p_values = np.linspace(0.001, 0.999, 100)

gini_values = [gini([p * 100, (1 - p) * 100]) for p in p_values]
entropy_values = [entropy([p * 100, (1 - p) * 100]) for p in p_values]
error_values = [classification_error([p * 100, (1 - p) * 100]) for p in p_values]

plt.figure(figsize=(10, 6))
plt.plot(p_values, gini_values, 'b-', linewidth=2, label='Gini')
plt.plot(p_values, entropy_values, 'r-', linewidth=2, label='Entropie')
plt.plot(p_values, error_values, 'g-', linewidth=2, label='Erreur de classification')

for name, counts, g, e, c in results:
    p = counts[0] / sum(counts)
    plt.scatter([p], [g], color='blue', s=100, zorder=5)
    plt.scatter([p], [e], color='red', s=100, zorder=5)
    plt.scatter([p], [c], color='green', s=100, zorder=5)

plt.xlabel('Proportion de la classe positive (p)', fontsize=12)
plt.ylabel('Impurete', fontsize=12)
plt.title('Comparaison des mesures d\'impurete', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.ylim(0, 1.1)
plt.show()


---
# Partie 2 - Implementation d'un mini-arbre de decision "from scratch"

## 2.1 Jeu de donnees pedagogique - Credit simplifie


In [ ]:
# Dataset: decision d'accorder un credit
data = np.array([
    [1, 1, 50, 0],  # Proprio, Matrimonial, Revenu, Defaut
    [0, 0, 30, 1],
    [1, 2, 45, 0],
    [0, 1, 35, 0],
    [1, 0, 60, 0],
    [0, 0, 25, 1],
    [1, 1, 55, 1],
    [0, 2, 40, 0],
    [1, 0, 70, 0],
    [0, 1, 28, 1],
    [1, 2, 48, 0],
    [0, 0, 32, 0],
])

labels = np.array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0])

feature_names = ['Proprietaire', 'Etat_Matrimonial', 'Revenu', 'Defaut_Precedent']
class_names = ['Refuse', 'Accorde']

print("Dataset Credit Simplifie")
print(f"Nombre d'exemples: {len(data)}")
print(f"Distribution des classes: {Counter(labels)}")


## 2.2 Structure de noeud et fonction de meilleur split

In [ ]:
def get_class_counts(y):
    counts = [0, 0]
    for label in y:
        counts[label] += 1
    return counts

class Node:
    def __init__(self):
        self.is_leaf = False
        self.prediction = None
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.gini = None
        self.samples = None

def find_best_split(X, y, feature_types):
    n_samples, n_features = X.shape
    parent_gini = gini(get_class_counts(y))
    
    best_gain = 0
    best_feature = None
    best_threshold = None
    
    for feature_idx in range(n_features):
        values = X[:, feature_idx]
        
        if feature_types[feature_idx] == 'continuous':
            sorted_values = np.unique(values)
            thresholds = (sorted_values[:-1] + sorted_values[1:]) / 2
            
            for thresh in thresholds:
                left_mask = values <= thresh
                right_mask = ~left_mask
                
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                left_gini = gini(get_class_counts(y[left_mask]))
                right_gini = gini(get_class_counts(y[right_mask]))
                
                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)
                weighted_gini = (n_left * left_gini + n_right * right_gini) / n_samples
                
                gain = parent_gini - weighted_gini
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = thresh
        else:
            unique_values = np.unique(values)
            for val in unique_values:
                left_mask = values == val
                right_mask = ~left_mask
                
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                left_gini = gini(get_class_counts(y[left_mask]))
                right_gini = gini(get_class_counts(y[right_mask]))
                
                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)
                weighted_gini = (n_left * left_gini + n_right * right_gini) / n_samples
                
                gain = parent_gini - weighted_gini
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = val
    
    return best_feature, best_threshold, best_gain


## 2.3 Construction recursive de l'arbre

In [ ]:
def build_tree(X, y, feature_types, depth=0, max_depth=None, min_samples_leaf=1):
    node = Node()
    node.samples = len(y)
    node.gini = gini(get_class_counts(y))
    
    # Condition d'arret 1: noeud pur
    if len(np.unique(y)) == 1:
        node.is_leaf = True
        node.prediction = y[0]
        return node
    
    # Condition d'arret 2: profondeur maximale
    if max_depth is not None and depth >= max_depth:
        node.is_leaf = True
        node.prediction = Counter(y).most_common(1)[0][0]
        return node
    
    # Condition d'arret 3: pas assez d'exemples
    if len(y) < 2 * min_samples_leaf:
        node.is_leaf = True
        node.prediction = Counter(y).most_common(1)[0][0]
        return node
    
    best_feature, best_threshold, best_gain = find_best_split(X, y, feature_types)
    
    if best_feature is None or best_gain <= 0:
        node.is_leaf = True
        node.prediction = Counter(y).most_common(1)[0][0]
        return node
    
    node.feature_index = best_feature
    node.threshold = best_threshold
    
    if feature_types[best_feature] == 'continuous':
        left_mask = X[:, best_feature] <= best_threshold
    else:
        left_mask = X[:, best_feature] == best_threshold
    
    right_mask = ~left_mask
    
    if np.sum(left_mask) < min_samples_leaf or np.sum(right_mask) < min_samples_leaf:
        node.is_leaf = True
        node.prediction = Counter(y).most_common(1)[0][0]
        return node
    
    node.left = build_tree(X[left_mask], y[left_mask], feature_types, depth + 1, max_depth, min_samples_leaf)
    node.right = build_tree(X[right_mask], y[right_mask], feature_types, depth + 1, max_depth, min_samples_leaf)
    
    return node


## 2.4 Fonction de prediction

In [ ]:
def predict_one(node, x, feature_types):
    if node.is_leaf:
        return node.prediction
    
    feature_val = x[node.feature_index]
    
    if feature_types[node.feature_index] == 'continuous':
        if feature_val <= node.threshold:
            return predict_one(node.left, x, feature_types)
        else:
            return predict_one(node.right, x, feature_types)
    else:
        if feature_val == node.threshold:
            return predict_one(node.left, x, feature_types)
        else:
            return predict_one(node.right, x, feature_types)

def predict(node, X, feature_types):
    return np.array([predict_one(node, x, feature_types) for x in X])

def print_tree(node, feature_names, feature_types, class_names, depth=0):
    indent = "  " * depth
    if node.is_leaf:
        print(f"{indent}-> Classe: {class_names[node.prediction]} (Gini: {node.gini:.3f}, n={node.samples})")
    else:
        feat_name = feature_names[node.feature_index]
        if feature_types[node.feature_index] == 'continuous':
            print(f"{indent}[{feat_name} <= {node.threshold:.1f}] (Gini: {node.gini:.3f}, n={node.samples})")
        else:
            print(f"{indent}[{feat_name} == {node.threshold}] (Gini: {node.gini:.3f}, n={node.samples})")
        print(f"{indent}Oui:")
        print_tree(node.left, feature_names, feature_types, class_names, depth + 1)
        print(f"{indent}Non:")
        print_tree(node.right, feature_names, feature_types, class_names, depth + 1)


### Entrainement et test

In [ ]:
feature_types = ['categorical', 'categorical', 'continuous', 'categorical']

tree = build_tree(data, labels, feature_types, max_depth=3, min_samples_leaf=1)

print("Structure de l'arbre:")
print("-" * 40)
print_tree(tree, feature_names, feature_types, class_names)

predictions = predict(tree, data, feature_types)
accuracy = np.mean(predictions == labels)
print(f"\nPrecision sur l'entrainement: {accuracy * 100:.1f}%")


## 2.5 Comparaison avec sklearn

In [ ]:
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(data, labels)

sklearn_predictions = clf.predict(data)
sklearn_accuracy = accuracy_score(labels, sklearn_predictions)

print(f"Precision mini-arbre:  {accuracy * 100:.1f}%")
print(f"Precision sklearn:     {sklearn_accuracy * 100:.1f}%")

print("\nMatrice de confusion (mini-arbre):")
print(confusion_matrix(labels, predictions))


---
# Partie 3 - Extensions : sur-apprentissage, elagage et forets aleatoires

## 3.1 Selection d'un jeu de donnees reel


In [ ]:
bc_data = load_breast_cancer()
X, y = bc_data.data, bc_data.target
bc_feature_names = bc_data.feature_names
target_names = bc_data.target_names

print(f"Dataset: Breast Cancer Wisconsin")
print(f"Nombre d'exemples: {len(X)}")
print(f"Nombre d'attributs: {len(bc_feature_names)}")
print(f"Classes: {list(target_names)}")
print(f"Distribution: {np.bincount(y)}")


## 3.2 Division train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
print(f"Taille train: {len(X_train)}, Taille test: {len(X_test)}")


## 3.3 & 3.4 Effet de la profondeur

In [ ]:
max_depths = [1, 2, 3, 4, 5, 7, 10, None]
train_scores = []
test_scores = []

for depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    clf.fit(X_train, y_train)
    train_scores.append(accuracy_score(y_train, clf.predict(X_train)))
    test_scores.append(accuracy_score(y_test, clf.predict(X_test)))

print(f"{'max_depth':<12} {'Train Acc':>12} {'Test Acc':>12} {'Ecart':>10}")
print("-" * 50)
for i, depth in enumerate(max_depths):
    ecart = train_scores[i] - test_scores[i]
    depth_str = str(depth) if depth else "None"
    print(f"{depth_str:<12} {train_scores[i]:>12.4f} {test_scores[i]:>12.4f} {ecart:>10.4f}")


In [ ]:
plt.figure(figsize=(10, 5))
x_labels = [str(d) if d else 'None' for d in max_depths]
x_pos = range(len(max_depths))

plt.plot(x_pos, train_scores, 'b-o', linewidth=2, markersize=8, label='Train')
plt.plot(x_pos, test_scores, 'r-o', linewidth=2, markersize=8, label='Test')
plt.xticks(x_pos, x_labels)
plt.xlabel('max_depth', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Effet de la profondeur sur les performances', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0.85, 1.02)
plt.show()


### Analyse du sur-apprentissage

**Observations:**
- Quand max_depth augmente, la precision sur le train augmente (jusqu'a 100%)
- La precision sur le test peut diminuer = **SUR-APPRENTISSAGE**
- Un grand ecart entre train et test indique un overfitting


## 3.5 Comparaison avec Random Forest

In [ ]:
n_estimators_values = [10, 50, 100, 200]

print(f"{'n_estimators':<15} {'Train Acc':>12} {'Test Acc':>12} {'F1-Score':>12}")
print("-" * 55)

for n_est in n_estimators_values:
    rf = RandomForestClassifier(n_estimators=n_est, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, rf.predict(X_train))
    test_acc = accuracy_score(y_test, rf.predict(X_test))
    f1 = f1_score(y_test, rf.predict(X_test))
    print(f"{n_est:<15} {train_acc:>12.4f} {test_acc:>12.4f} {f1:>12.4f}")


## 3.6 (Optionnel) AdaBoost

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)

best_tree = DecisionTreeClassifier(max_depth=4, random_state=42)
best_tree.fit(X_train, y_train)

best_rf = RandomForestClassifier(n_estimators=100, random_state=42)
best_rf.fit(X_train, y_train)

print("RESUME COMPARATIF")
print("=" * 50)
print(f"{'Methode':<30} {'Test Accuracy':>15}")
print("-" * 50)
print(f"{'Arbre simple (max_depth=4)':<30} {accuracy_score(y_test, best_tree.predict(X_test)):>15.4f}")
print(f"{'Random Forest (n=100)':<30} {accuracy_score(y_test, best_rf.predict(X_test)):>15.4f}")
print(f"{'AdaBoost (n=100)':<30} {accuracy_score(y_test, ada.predict(X_test)):>15.4f}")


---
# Partie 4 - Application metier et interpretation

## 4.1 Domaine d'application

**DOMAINE: DIAGNOSTIC MEDICAL (Cancer du sein)**

**Problematique metier:**
Le diagnostic precoce du cancer du sein est crucial. Les medecins utilisent des biopsies 
pour analyser les caracteristiques des cellules. L'objectif est de classifier les tumeurs en:
- BENIGNES (non cancereuses)
- MALIGNES (cancereuses)

**Variables explicatives:** 30 caracteristiques des cellules (rayon, texture, perimetre, etc.)

**Variable cible:** Type de tumeur (0=Maligne, 1=Benigne)


## 4.2 Modele final

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=4, min_samples_leaf=5, random_state=42)
tree_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
rf_model.fit(X_train, y_train)

print("Performances des modeles:")
print("-" * 50)
print(f"Arbre de decision: {accuracy_score(y_test, tree_model.predict(X_test)):.4f}")
print(f"Random Forest:     {accuracy_score(y_test, rf_model.predict(X_test)):.4f}")

print("\nRapport de classification (Arbre):")
print(classification_report(y_test, tree_model.predict(X_test), target_names=['Maligne', 'Benigne']))


## 4.3 Regles de decision

In [ ]:
print("Regles extraites de l'arbre:")
print("-" * 50)
print(export_text(tree_model, feature_names=list(bc_feature_names)))


In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(tree_model, feature_names=bc_feature_names, class_names=['Maligne', 'Benigne'],
          filled=True, rounded=True, fontsize=8)
plt.title("Arbre de decision pour le diagnostic du cancer du sein", fontsize=14)
plt.tight_layout()
plt.show()


### Importance des variables

In [ ]:
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

print("Top 10 variables les plus importantes:")
for i in range(10):
    print(f"  {i+1}. {bc_feature_names[indices[i]]}: {importances[indices[i]]:.4f}")


## 4.4 Discussion

### Interpretabilite (Arbres vs Forets)

| Critere | Arbre de decision | Random Forest |
|---------|-------------------|---------------|
| Interpretabilite | Tres bonne | Faible |
| Performance | Moyenne | Elevee |
| Sur-apprentissage | Risque eleve | Risque faible |
| Visualisation | Facile | Difficile |

### Limites observees
1. **Zones d'erreur**: Cas limites difficiles a classer
2. **Biais potentiels**: Dataset desequilibre
3. **Donnees manquantes**: Non gerees nativement
4. **Generalisation**: Depend de la population d'entrainement

### Recommandations
- Utiliser l'arbre comme outil d'aide, pas de remplacement du medecin
- Confirmer avec l'expertise medicale
- Actualiser regulierement avec de nouvelles donnees


In [ ]:
cm = confusion_matrix(y_test, tree_model.predict(X_test))
print("Matrice de confusion finale")
print("=" * 40)
print("              Predit")
print("           Maligne  Benigne")
print(f"Reel Maligne   {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"     Benigne   {cm[1,0]:3d}      {cm[1,1]:3d}")
print(f"\nAttention: {cm[0,1]} faux positifs (malignes classees benignes) = DANGEREUX")


---
# Conclusion

Ce projet a permis de:
1. Comprendre les mesures d'impurete (Gini, entropie, erreur)
2. Implementer un arbre de decision from scratch
3. Analyser le sur-apprentissage et les hyperparametres
4. Comparer arbres simples, Random Forest et AdaBoost
5. Appliquer ces modeles a un cas reel de diagnostic medical
6. Interpreter les regles pour un non-expert
